In [24]:
import sys
import os

# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [25]:
import pandas as pd
import pyarrow.parquet as pq
from pandas import DataFrame

In [26]:
def load_data(path):
    parquet_file = pq.ParquetFile(path)


    batch = next(parquet_file.iter_batches())
    df: DataFrame = batch.to_pandas()
    return df

df = load_data("../data/train/attributes.parquet")     
df

,variantid,categories,characteristic_attributes_mapping
0,47920382,"{""1"": ""EPG"", ""2"": ""Детские товары"", ""3"": ""Игру...","{""Цвет товара"": [""бежевый"", ""светло-розовый""],..."
1,49801845,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Количество в упаковке, шт"": [""1""], ""Бренд"": ..."
2,49853444,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Бренд"": [""Vervaco""], ""Тип"": [""Набор для выши..."
3,49893028,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""серый""], ""Ширина, см"": [""0.8..."
4,49987483,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""разноцветный""], ""Название цв..."
...,...,...,...
65531,1538894418,"{""1"": ""Книги и цифровые книги"", ""2"": ""Книги"", ...","{""ISBN"": [""978-5-00167-488-7""], ""Количество ст..."
65532,1539114323,"{""1"": ""EPG"", ""2"": ""Строительство и ремонт"", ""3...","{""Комплектация"": [""рамка""], ""Гарантия"": [""1 го..."
65533,1539195769,"{""1"": ""EPG"", ""2"": ""Бытовая химия"", ""3"": ""Моющи...","{""Комплектация"": [""средство 1 штука, 1 тряпка ..."
65534,1539274152,"{""1"": ""EPG"", ""2"": ""Продукты питания"", ""3"": ""Со...","{""Степень газирования"": [""Сильногазированная""]..."


In [28]:
import src.transformations as tf
from transformers import BertModel, BertTokenizer

df = load_data('../data/train/attributes.parquet')

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

tf.attr_transformer().transform(df)
tf.categories_transformer().transform(df)

df

/home/sv-cheats-1/Documents/PROJECTS/matching-tovarov-5699/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,variantid,categories,characteristic_attributes_mapping
0,47920382,Игрушки Игрушка для ванной Детские товары,"Бренд:Funny Ducks;Вес товара, г:50;Код ТРУ:320..."
1,49801845,"Хобби и творчество Стеклярус, бусины, стразы М...","Бренд:Нет бренда;Вес товара, г:20;Количество в..."
2,49853444,Набор для вышивания Хобби и творчество Набор д...,Бренд:Vervaco;Возрастные ограничения:От 7 лет;...
3,49893028,"Хобби и творчество Нитки, пряжа Тесьма, кружев...","Бренд:Prym;Длина, м:2;Состав ниток:Ткань;Стран..."
4,49987483,Хобби и творчество Настольная игра Настольные ...,Бренд:Оригами;Вид выпуска товара:Фабричное про...
...,...,...,...
65531,1538894418,Книги Книги и цифровые книги Печатная книга: П...,ISBN:978-5-00167-488-7;Автор на обложке:Ивамур...
65532,1539114323,Рамка электроустановочная Электроустановочные ...,Бренд:MOiO ТВОЙ УМНЫЙ ДОМ;Высота товара в упак...
65533,1539195769,"Средство чистящее для ковров, обивки, кожи, ме...",Бренд:PROSEPT;Единиц в одном товаре:2;Комплект...
65534,1539274152,"Газированный напиток Продукты питания Соки, во...",Бренд:Добрый;Вкус (лимонад):Вишня;Единиц в одн...


1.
Бренд    
Название бренда    
Возрастные ограничения   


эмбедим эту шелупонь

2. 5х5 аттрибутов
сравниваем эмбеддинги ключей
если > threshold -> сравниваем значения

3. считаем score [-1,1]


In [29]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sergeyzh/rubert-tiny-turbo')

In [30]:
from sklearn.decomposition import PCA
import torch
import random
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin


class bert_64_transformer(BaseEstimator, TransformerMixin):
    def __init__(self, model, feature: str):
        super().__init__()

        self.model = model
        self.feature = feature
        self.pca = PCA(n_components=64)

        # self.device = "cuda" if torch.cuda.is_available() else "cpu"
        # self.projection_layer = nn.Linear(768, 64).to(self.device)

    def transform(self, X):
        # Fix seeds for reproducibility
        def set_seed(seed):
            torch.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            np.random.seed(seed)
            random.seed(seed)
            torch.backends.cudnn.deterministic = True

        set_seed(42)
        
        # Apply the encoding function to the specified feature
        X[self.feature] = list(self.model.encode(X[self.feature]))

        return X


In [31]:
bert_64_transformer(model, 'categories').transform(df)
df

,variantid,categories,characteristic_attributes_mapping
0,47920382,"[0.01180572, 0.013707614, 0.050783817, -0.0464...","Бренд:Funny Ducks;Вес товара, г:50;Код ТРУ:320..."
1,49801845,"[0.07651598, 0.020285651, 0.021583704, 0.00165...","Бренд:Нет бренда;Вес товара, г:20;Количество в..."
2,49853444,"[0.083931595, 0.029981451, 0.03453764, -0.0053...",Бренд:Vervaco;Возрастные ограничения:От 7 лет;...
3,49893028,"[0.058669478, 0.024018902, -0.020405347, -0.02...","Бренд:Prym;Длина, м:2;Состав ниток:Ткань;Стран..."
4,49987483,"[0.087514356, 0.025321718, 0.039086368, -0.060...",Бренд:Оригами;Вид выпуска товара:Фабричное про...
...,...,...,...
65531,1538894418,"[0.06241672, -0.00071906525, 0.035121743, -0.0...",ISBN:978-5-00167-488-7;Автор на обложке:Ивамур...
65532,1539114323,"[-0.011476818, 0.05846518, 0.06270081, -0.0598...",Бренд:MOiO ТВОЙ УМНЫЙ ДОМ;Высота товара в упак...
65533,1539195769,"[0.029886225, 0.027343074, 0.048641473, -0.028...",Бренд:PROSEPT;Единиц в одном товаре:2;Комплект...
65534,1539274152,"[0.03712005, -0.022005554, 0.007981729, -0.044...",Бренд:Добрый;Вкус (лимонад):Вишня;Единиц в одн...


In [32]:
len(df["categories"].iloc[0])

312

In [33]:
# tf.bert_64_transformer(model, "characteristic_attributes_mapping").transform(
#     df
# )
# df

,variantid,categories,characteristic_attributes_mapping
0,47920382,"[0.01180572, 0.013707614, 0.050783817, -0.0464...",0.030944
1,49801845,"[0.07651598, 0.020285651, 0.021583704, 0.00165...",-0.063087
2,49853444,"[0.083931595, 0.029981451, 0.03453764, -0.0053...",0.026884
3,49893028,"[0.058669478, 0.024018902, -0.020405347, -0.02...",0.044328
4,49987483,"[0.087514356, 0.025321718, 0.039086368, -0.060...",-0.021138
...,...,...,...
65531,1538894418,"[0.06241672, -0.00071906525, 0.035121743, -0.0...",-0.352550
65532,1539114323,"[-0.011476818, 0.05846518, 0.06270081, -0.0598...",0.028176
65533,1539195769,"[0.029886225, 0.027343074, 0.048641473, -0.028...",-0.031204
65534,1539274152,"[0.03712005, -0.022005554, 0.007981729, -0.044...",-0.048467


In [50]:
torch.from_numpy(df[df['variantid'] == 1538894418]['categories'].values[0])

tensor([ 6.2417e-02, -7.1907e-04,  3.5122e-02, -1.7937e-02,  4.1357e-02,
         6.8176e-02, -1.4272e-02,  5.7042e-03,  1.9830e-02, -1.5672e-02,
         4.5193e-02,  8.4497e-05,  1.1300e-02, -1.2617e-02,  5.9591e-02,
        -4.5765e-02, -5.6827e-03, -3.4521e-03, -3.1580e-02, -1.1409e-02,
        -2.5635e-02,  4.6035e-02,  4.4940e-02, -1.1818e-02,  1.0125e-01,
         1.2720e-02, -2.8757e-02, -1.7469e-02, -3.4320e-02,  9.8061e-03,
        -1.1998e-02, -7.3433e-03, -1.4762e-02, -5.4347e-02, -1.5105e-02,
        -4.2240e-02,  3.6362e-02, -7.0121e-03,  9.8235e-02, -5.1154e-02,
        -2.5867e-02,  1.3674e-02, -2.9398e-03, -4.1738e-02,  2.2118e-02,
        -4.0994e-02, -9.7656e-03,  1.7584e-02,  1.1803e-03,  2.3664e-02,
        -1.3109e-03, -2.5150e-02,  8.3686e-03,  5.7831e-03, -1.8358e-02,
        -3.2950e-03, -3.7469e-02,  6.2283e-02, -7.2418e-03,  3.8305e-02,
        -1.3069e-02, -5.0829e-03, -1.5891e-03, -5.9412e-03, -2.5323e-02,
         3.0819e-02,  4.6544e-03,  4.6215e-02,  4.0

In [56]:
from torch import cosine_similarity


x1 = torch.from_numpy(df[df['variantid'] == 1538894418]['categories'].values[0]).unsqueeze(0)
x2 = torch.from_numpy(df[df['variantid'] == 1539114323]['categories'].values[0]).unsqueeze(0)

cosine_similarity(x1, x2)

tensor([0.6914])

In [62]:
dff = load_data('../data/train/attributes.parquet')
dff[dff['variantid'] == 1538894418]['categories']
dff[dff['variantid'] == 1539114323]['categories']

65532    {"1": "EPG", "2": "Строительство и ремонт", "3...
Name: categories, dtype: object

In [57]:
cosine_similarity(x1, x2)

tensor([0.6914])

In [67]:
df_train = pd.read_parquet('../data/train/train.parquet')
df_train

,variantid1,variantid2,target
0,1447875869,1447872068,1
1,1176231201,284733670,1
2,658617865,549848659,0
3,719320625,719370486,1
4,1067645658,949954740,0
...,...,...,...
1168511,970087885,944020046,0
1168512,834554806,118393780,1
1168513,1473905394,1473908406,1
1168514,104892989,1549506903,1


In [77]:

ids = [
47920382	,
49801845	,
49853444	,
49893028	,
49987483	,
1538894418	,
1539114323	,
1539195769	,
1539274152	,
1539290656,
]
for id in ids:
    print(df_train[(df_train['target'] == 1) & ((df_train['variantid1'] == id) | (df_train['variantid2'] == id))])

Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
        variantid1  variantid2  target
491153  1052580069  1539274152       1
Empty DataFrame
Columns: [variantid1, variantid2, target]
Index: []
